In [1]:
from datetime import datetime
import numpy as np, pandas as pd

!pip install talos
!pip install spacy

import re
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.tokenize import word_tokenize
import nltk
nltk.download('all')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download("punkt")
import string
from string import ascii_lowercase

from tqdm import tqdm_notebook
import itertools
import io

import matplotlib.pyplot as plt
%matplotlib inline

from functools import reduce
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
import talos

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\shrav\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\shrav\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\shrav\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\shrav\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\shrav\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [2]:
dt = datetime.now()
print(dt)
train_data = pd.read_csv('C:/Users/shrav/Desktop/Fall Sem/ML/project/Train/train.csv')

2022-11-29 16:37:40.604436


In [3]:
dt = datetime.now()
print(dt)

test_data=pd.read_csv('C:/Users/shrav/Desktop/Fall Sem/ML/project/Train/test.csv')
test_data.head()

2022-11-29 16:37:42.616644


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [4]:
train_data.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [5]:
test_data.isnull().any()

id              False
comment_text    False
dtype: bool

In [6]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
y = train_data[labels].values

In [7]:
Re_words = {
    ' american ':
        [
            'amerikan'
        ],

    ' adolf ':
        [
            'adolf'
        ],


    ' hitler ':
        [
            'hitler'
        ],

    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 'f**k','fu*k',
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$',
            '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s','a55', '@$$'
        ],

    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole', 'a**hole'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h', 'b!tch', 'bi+ch', 'l3itch'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' trans gender':
        [
            'transgender'
        ],

    ' gay ':
        [
            'gay'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k', 'dik'
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],

    ' homo sex ual':
        [
            'homosexual'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots'
                                                                                      'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t', '$h1t'
        ],

    ' shit hole ':
        [
            'shythole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],

    ' ass head':
        [
            'butthead'
        ],

    ' sex ':
        [
            'sexy', 's3x', 'sexuality'
        ],


    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            'stfu', 'st*u'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses', 'p*ssy'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' mother fucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
    ' fucking ':
        [
            'f*$%-ing'
        ],
}


In [8]:
def text_cleaning(words,delete_repeated=True, delete_word_pat=True, is_lower=True):

  if is_lower:
    words=words.lower()
    
  if delete_word_pat:
    for target, patterns in Re_words.items():
      for pat in patterns:
        words=str(words).replace(pat, target)

  if delete_repeated:
    words = re.sub(r'(.)\1{2,}', r'\1', words) 

  words = str(words).replace("\n", " ")
  words = re.sub(r'[^\w\s]',' ',words)
  words = re.sub('[0-9]',"",words)
  words = re.sub(" +", " ", words)
  words = re.sub("([^\x00-\x7F])+"," ",words)
  return words 

In [9]:
train_data['comment_text']=train_data['comment_text'].apply(lambda x: text_cleaning(x))

In [10]:
test_data['comment_text']=test_data['comment_text'].apply(lambda x: text_cleaning(x))

In [11]:
train_comments=train_data['comment_text']
test_comments=test_data['comment_text']

In [12]:
train_comments=list(train_comments)
test_comments=list(test_comments)

In [13]:
wordnet = WordNetLemmatizer()

def lemmatizer(words, lemmatization=True):
  op=""
  if lemmatization:
    words=words.split(" ")
    for word in words:
       word1 = wordnet.lemmatize(word, pos = "n")
       word2 = wordnet.lemmatize(word1, pos = "v")
       word3 = wordnet.lemmatize(word2, pos = "a")
       word4 = wordnet.lemmatize(word3, pos = "r")
       op=op + " " + word4
  else:
    op=words
  
  return str(op.strip()) 

In [14]:
converted_trained_data= [] 

for line in tqdm_notebook(train_comments, total=159571): 
    converted_trained_data.append(lemmatizer(line))


  0%|          | 0/159571 [00:00<?, ?it/s]

In [15]:
converted_tested_data = [] 

for line in tqdm_notebook(test_comments, total=len(test_comments)): 
    converted_tested_data.append(lemmatizer(line))


  0%|          | 0/153164 [00:00<?, ?it/s]

In [16]:
list_of_stopwords=STOP_WORDS

def processing_Strings():
    for size in itertools.count(1):
        for s in itertools.product(ascii_lowercase, repeat=size):
            yield "".join(s)

list_of_texts=[]
for s in processing_Strings():
    list_of_texts.append(s)
    if s == 'zz':
        break

In [17]:
list_of_texts.remove('i')
list_of_texts.remove('a')
list_of_texts.remove('am')
list_of_texts.remove('an')
list_of_texts.remove('as')
list_of_texts.remove('at')
list_of_texts.remove('be')
list_of_texts.remove('by')
list_of_texts.remove('do')
list_of_texts.remove('go')
list_of_texts.remove('he')
list_of_texts.remove('hi')
list_of_texts.remove('if')
list_of_texts.remove('is')
list_of_texts.remove('in')
list_of_texts.remove('me')
list_of_texts.remove('my')
list_of_texts.remove('no')
list_of_texts.remove('of')
list_of_texts.remove('on')
list_of_texts.remove('or')
list_of_texts.remove('ok')
list_of_texts.remove('so')
list_of_texts.remove('to')
list_of_texts.remove('up')
list_of_texts.remove('us')
list_of_texts.remove('we')

In [18]:
for letter in list_of_texts:
    list_of_stopwords.add(letter)
print("These words added to stopwords")

These words added to stopwords


In [19]:
def find_stopwords(text, find_stopword=True):
  op=""
  if find_stopword:
    text=text.split(" ")
    for word in text:
      if not word in list_of_stopwords:
        op=op+" "+word 
  else:
    op=text

  return str(op.strip())  

stop_words = [] 

for line in tqdm_notebook(converted_trained_data, total=159571): 
    stop_words.append(find_stopwords(line))

  0%|          | 0/159571 [00:00<?, ?it/s]

In [20]:
def combine_strings1(stopword):
  string1=""
  for item in range(39893):
    string1=string1+" "+stopword[item]
  return string1

def combine_strings2(stopword):
  string2=""
  for item in range(39893,79785):
    string2=string2+" "+stopword[item]
  return string2

def combine_strings3(stopword):
  string3=""
  for item in range(79785,119678):
    string3=string3+" "+stopword[item]
  return string3

def combine_strings4(stopword):
  string4=""
  for item in range(119678,159571):
    string4=string4+" "+stopword[item]
  return string4

In [21]:
final_strings1=combine_strings1(stop_words)
final_strings2=combine_strings2(stop_words)
final_strings3=combine_strings3(stop_words)
final_strings4=combine_strings4(stop_words)

In [22]:
def string_words(str):
    add = dict()
    texts = str.split()

    for word in texts:
        if word in add:
            add[word] += 1
        else:
            add[word] = 1

    return add

In [23]:
string_dict1=string_words(final_strings1)
string_dict2=string_words(final_strings2)
string_dict3=string_words(final_strings3)
string_dict4=string_words(final_strings4)

In [24]:
total_string_dict1 = pd.DataFrame(list(string_dict1.items()),columns = ['Word','Count'])
total_string_dict2 = pd.DataFrame(list(string_dict2.items()),columns = ['Word','Count'])
total_string_dict3 = pd.DataFrame(list(string_dict3.items()),columns = ['Word','Count'])
total_string_dict4 = pd.DataFrame(list(string_dict4.items()),columns = ['Word','Count'])

In [25]:
top50_stop_words1=total_string_dict1.sort_values(by=['Count'],ascending=False).head(50)
top50_stop_words2=total_string_dict2.sort_values(by=['Count'],ascending=False).head(50)
top50_stop_words3=total_string_dict3.sort_values(by=['Count'],ascending=False).head(50)
top50_stop_words4=total_string_dict4.sort_values(by=['Count'],ascending=False).head(50)

In [26]:
common_stop_words=list(reduce(set.intersection,map(set,[top50_stop_words1.Word,top50_stop_words2.Word,top50_stop_words3.Word,top50_stop_words4.Word])))

In [27]:
stop_words=common_stop_words

In [28]:
for word in stop_words:
    list_of_stopwords.add(word)
print("we have successfully added the common stop words into list of stopwords!!")

we have successfully added the common stop words into list of stopwords!!


In [29]:
def delete_stoword_data(words, delete=True):
  op = ""
  if delete:
    words=words.split(" ")
    for word in words:
      if word not in list_of_stopwords:
        op=op + " " + word
  else :
    op=words

  return str(op.strip())      

In [30]:
preprocessed_data = [] 

for line in tqdm_notebook(converted_trained_data, total=159571): 
    preprocessed_data.append(delete_stoword_data(line))

  0%|          | 0/159571 [00:00<?, ?it/s]

In [31]:
preprocessed_data[152458]

'agree graemel intention revert nazi protect report revert rule notice board post report revert nazi graemel revert nazi unacceptable site admins feel unjustly revert hour period report revert noticeboard'

In [32]:
preprocessed_data2 = [] 

for line in tqdm_notebook(converted_tested_data, total=153164): 
    preprocessed_data2.append(delete_stoword_data(line))

  0%|          | 0/153164 [00:00<?, ?it/s]

In [33]:
data_length_max=100000      
total_pad = 200          
split_data = 0.2      
Pre_trained_length = 300

In [34]:
word_list = Tokenizer(num_words=data_length_max)
word_list.fit_on_texts(list(preprocessed_data))
trained_words = word_list.texts_to_sequences(preprocessed_data)
tested_words = word_list.texts_to_sequences(preprocessed_data2)

In [35]:
word_index=word_list.word_index
print("Words in Vocabulary: ",len(word_index))

Words in Vocabulary:  148917


In [36]:
token_data_train=pad_sequences(trained_words, maxlen=total_pad, padding = 'post')
token_data_test=pad_sequences(tested_words, maxlen=total_pad, padding = 'post')

In [37]:
print('Tokenized sentences: \n', token_data_train[10])

Tokenized sentences: 
 [   93   555 32193   216   169 32193   216    67   556    93   343   555
  1059    93   315  5335    93    96   364   246   343   555  1319    93
   364     9    93   555   169    93   149    23    61   556    93   555
   465    82  1064   380  1050  2379   466    13    93   169   384   343
   232   169   219   127    86     6   149    54   234  4519 32193   216
   169 32193   216    67   148   364   289    45   556    11    21    54
   400  1528    11   148   556  1246    54  1616   111   111  4650   111
    71    21   542  1233    54  1316    95   111  1555    54  1112   484
    54   411     8   148    45    11   145  2185   426     8   292  1507
    47   149   213   127    93     8     8    54     8    93    54     8
    54     8   169   148    23    61   556     8   148   169    57   839
  3104   232     8   219   127    86    54   101    44   411    93   334
     6   149    54   234     0     0     0     0     0     0     0     0
     0     0     0     0    

In [38]:
index = np.arange(token_data_train.shape[0])
np.random.shuffle(index)

In [39]:
token_data_train = token_data_train[index]
labels = y[index]

In [40]:
validated_samples = int(split_data*token_data_train.shape[0])
trained_tokens1 = token_data_train[: -validated_samples]
trained_tokens2 = labels[: -validated_samples]
x_val = token_data_train[-validated_samples: ]
y_val = labels[-validated_samples: ]

In [41]:
Pre_trained_data = {}
f = open('C:/Users/shrav/Desktop/Fall Sem/ML/project/Train/wiki-news-300d-1M.vec', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    Pre_trained_data[word] = np.asarray(values[1:], dtype='float32')
f.close()

In [42]:
Pre_trained_data2 = np.random.random((len(word_index) + 1, Pre_trained_length))
for word, i in word_index.items():
    text_vector = Pre_trained_data.get(word)
    if text_vector is not None:
        Pre_trained_data2[i] = text_vector
print(" Completed!")

 Completed!


In [46]:

dt = datetime.now()
print(dt)

def LSTM_Model_classifier(trained_tokens1,trained_tokens2,x_val,y_val,params):

  inp=Input(shape=(total_pad, ),dtype='int32')

  lstm_layer1 = Embedding(len(word_index) + 1,
                           Pre_trained_length,
                           weights = [Pre_trained_data2],
                           input_length = total_pad,
                           trainable=False,
                           name = 'embeddings')
  embedded_sequences = lstm_layer1(inp)

  x = LSTM(params['output_count_lstm'], return_sequences=True,name='lstm_layer')(embedded_sequences)
  
  x = GlobalMaxPool1D()(x)
  
  x = Dropout(params['dropout'])(x)
  
  x = Dense(params['output_count_dense'], activation=params['activation'], kernel_initializer='he_uniform')(x)
  
  x = Dropout(params['dropout'])(x)
  
  preds = Dense(6, activation=params['last_activation'], kernel_initializer='glorot_uniform')(x)

  model = Model(inputs=inp, outputs=preds)

  model.compile(loss=params['loss'], optimizer=params['optimizer'], metrics=['accuracy'])

  model_info=model.fit(trained_tokens1,trained_tokens2, epochs=params['epochs'], batch_size=params['batch_size'],  validation_data=(x_val, y_val))

  return model_info, model


2022-11-29 17:03:32.289447


In [49]:
p={
    'output_count_lstm': [40,50,60],
    'output_count_dense': [30,40,50],
    'batch_size': [32],
    'epochs':[3],
    'optimizer':['adam'],
    'activation':['relu'],
    'last_activation': ['sigmoid'],
    'dropout':[0.1,0.2],
    'loss': ['binary_crossentropy']   
}

In [50]:
dt = datetime.now()
print(dt)

scan_results = talos.Scan(x=trained_tokens1,
               y=trained_tokens2,
               x_val=x_val,
               y_val=y_val,
               model=LSTM_Model_classifier,
               params=p,
               experiment_name='lstm_params',
               print_params=True)

2022-11-29 17:08:49.583119


  0%|                                                                                           | 0/18 [00:00<?, ?it/s]

{'output_count_lstm': 40, 'output_count_dense': 30, 'batch_size': 32, 'epochs': 3, 'optimizer': 'adam', 'activation': 'relu', 'last_activation': 'sigmoid', 'dropout': 0.1, 'loss': 'binary_crossentropy'}
Epoch 1/3
3990/3990 [==============================] - 554s 138ms/step - loss: 0.0766 - accuracy: 0.8425 - val_loss: 0.0613 - val_accuracy: 0.9865
Epoch 2/3
3990/3990 [==============================] - 557s 140ms/step - loss: 0.0623 - accuracy: 0.9141 - val_loss: 0.0581 - val_accuracy: 0.9916
Epoch 3/3
3990/3990 [==============================] - 554s 139ms/step - loss: 0.0594 - accuracy: 0.9253 - val_loss: 0.0575 - val_accuracy: 0.9794


  6%|████▍                                                                          | 1/18 [27:51<7:53:37, 1671.62s/it]

{'output_count_lstm': 40, 'output_count_dense': 30, 'batch_size': 32, 'epochs': 3, 'optimizer': 'adam', 'activation': 'relu', 'last_activation': 'sigmoid', 'dropout': 0.2, 'loss': 'binary_crossentropy'}
Epoch 1/3
3990/3990 [==============================] - 559s 139ms/step - loss: 0.0775 - accuracy: 0.8654 - val_loss: 0.0608 - val_accuracy: 0.9938
Epoch 2/3
3990/3990 [==============================] - 554s 139ms/step - loss: 0.0642 - accuracy: 0.9008 - val_loss: 0.0595 - val_accuracy: 0.9907
Epoch 3/3
3990/3990 [==============================] - 552s 138ms/step - loss: 0.0614 - accuracy: 0.9117 - val_loss: 0.0585 - val_accuracy: 0.9938


...

 89%|███████████████████████████████████████████████████████████████████▌        | 16/18 [9:06:43<2:01:45, 3652.76s/it]

{'output_count_lstm': 60, 'output_count_dense': 50, 'batch_size': 32, 'epochs': 3, 'optimizer': 'adam', 'activation': 'relu', 'last_activation': 'sigmoid', 'dropout': 0.1, 'loss': 'binary_crossentropy'}
Epoch 1/3
3990/3990 [==============================] - 2049s 513ms/step - loss: 0.0730 - accuracy: 0.8606 - val_loss: 0.0603 - val_accuracy: 0.9938
Epoch 2/3
3990/3990 [==============================] - 2241s 562ms/step - loss: 0.0609 - accuracy: 0.9021 - val_loss: 0.0574 - val_accuracy: 0.9938
Epoch 3/3
3990/3990 [==============================] - 1106s 277ms/step - loss: 0.0580 - accuracy: 0.8953 - val_loss: 0.0577 - val_accuracy: 0.9938


 94%|██████████████████████████████████████████████████████████████████████▊    | 17/18 [10:36:42<1:09:37, 4177.95s/it]

{'output_count_lstm': 60, 'output_count_dense': 50, 'batch_size': 32, 'epochs': 3, 'optimizer': 'adam', 'activation': 'relu', 'last_activation': 'sigmoid', 'dropout': 0.2, 'loss': 'binary_crossentropy'}
Epoch 1/3
3990/3990 [==============================] - 1060s 265ms/step - loss: 0.0765 - accuracy: 0.8692 - val_loss: 0.0608 - val_accuracy: 0.9938
Epoch 2/3
3990/3990 [==============================] - 1092s 274ms/step - loss: 0.0627 - accuracy: 0.9053 - val_loss: 0.0586 - val_accuracy: 0.9901
Epoch 3/3
3990/3990 [==============================] - 1112s 279ms/step - loss: 0.0598 - accuracy: 0.9171 - val_loss: 0.0588 - val_accuracy: 0.9907


100%|█████████████████████████████████████████████████████████████████████████████| 18/18 [11:31:09<00:00, 2303.84s/it]


In [51]:
inp=Input(shape=(total_pad, ),dtype='int32')
lstm_layer1 = Embedding(len(word_index) + 1,
                           Pre_trained_length,
                           weights = [Pre_trained_data2],
                           input_length = total_pad,
                           trainable=False,
                           name = 'embeddings')
embedded_sequences = lstm_layer1(inp)
x = LSTM(50, return_sequences=True,name='lstm_layer')(embedded_sequences)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu", kernel_initializer='he_uniform')(x)
x = Dropout(0.1)(x)
preds = Dense(6, activation="sigmoid", kernel_initializer='glorot_uniform')(x)
model_1 = Model(inputs=inp, outputs=preds)
model_1.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [52]:
dt = datetime.now()
print(dt)
model_1.summary()

2022-11-30 08:01:34.818730
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 embeddings (Embedding)      (None, 200, 300)          44675400  
                                                                 
 lstm_layer (LSTM)           (None, 200, 50)           70200     
                                                                 
 global_max_pooling1d (Globa  (None, 50)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 50)                2550      
                                  

In [55]:
dt = datetime.now()
print(dt)

model_info_1=model_1.fit(trained_tokens1,trained_tokens2, epochs=3, batch_size=32,  validation_data=(x_val, y_val))

2022-11-30 10:30:07.505225
Epoch 1/3
3990/3990 [==============================] - 2257s 566ms/step - loss: 0.0516 - accuracy: 0.8704 - val_loss: 0.0567 - val_accuracy: 0.9424
Epoch 2/3
3990/3990 [==============================] - 1455s 365ms/step - loss: 0.0497 - accuracy: 0.8949 - val_loss: 0.0582 - val_accuracy: 0.9866
Epoch 3/3
3990/3990 [==============================] - 1999s 501ms/step - loss: 0.0481 - accuracy: 0.8935 - val_loss: 0.0584 - val_accuracy: 0.9345


In [56]:
model_1.save(filepath="C:/Users/shrav/Desktop/Fall Sem/ML/project/Results_LSTM/")

INFO:tensorflow:Assets written to: C:/Users/shrav/Desktop/Fall Sem/ML/project/Results_LSTM/assets


INFO:tensorflow:Assets written to: C:/Users/shrav/Desktop/Fall Sem/ML/project/Results_LSTM/assets


In [57]:
loaded_model_1 = keras.models.load_model(filepath="C:/Users/shrav/Desktop/Fall Sem/ML/project/Results_LSTM/")

In [58]:
dt = datetime.now()
print(dt)

test_values_1 = loaded_model_1.predict([token_data_test], batch_size=1, verbose=1)

2022-11-30 12:05:48.792744
153164/153164 [==============================] - 1832s 12ms/step


In [60]:
dt = datetime.now()
print(dt)

sample_submission = pd.read_csv('C:/Users/shrav/Desktop/Fall Sem/ML/project/Train/test_labels.csv')
test_values_1=pd.DataFrame(test_values_1,columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
submission = pd.DataFrame(sample_submission["id"])
combined_submission=pd.concat([submission,test_values_1],axis=1)
combined_submission.to_csv('C:/Users/shrav/Desktop/Fall Sem/ML/project/Results_LSTM/lstm_results.csv', index=False)

2022-11-30 12:38:43.317183


In [61]:
def toxicity_level(string):
    new_string = [string]
    new_string = word_list.texts_to_sequences(new_string)
    new_string = pad_sequences(new_string, maxlen=total_pad, padding='post')
    
    prediction = model_1.predict(new_string) 
    print("Toxicity levels for '{}':".format(string))
    print('Toxic:         {:.0%}'.format(prediction[0][0]))
    print('Severe Toxic:  {:.0%}'.format(prediction[0][1]))
    print('Obscene:       {:.0%}'.format(prediction[0][2]))
    print('Threat:        {:.0%}'.format(prediction[0][3]))
    print('Insult:        {:.0%}'.format(prediction[0][4]))
    print('Identity Hate: {:.0%}'.format(prediction[0][5]))
    print()
    
    return

In [62]:
dt = datetime.now()
print(dt)

toxicity_level('go jump off a bridge jerk')

2022-11-30 12:38:58.921800
1/1 [==============================] - 1s 934ms/step
Toxicity levels for 'go jump off a bridge jerk':
Toxic:         86%
Severe Toxic:  5%
Obscene:       43%
Threat:        1%
Insult:        63%
Identity Hate: 0%



In [63]:
toxicity_level('fuck ofF!!')

1/1 [==============================] - 0s 46ms/step
Toxicity levels for 'fuck ofF!!':
Toxic:         99%
Severe Toxic:  28%
Obscene:       96%
Threat:        0%
Insult:        74%
Identity Hate: 0%



In [64]:
toxicity_level('I Love you')

1/1 [==============================] - 0s 36ms/step
Toxicity levels for 'I Love you':
Toxic:         3%
Severe Toxic:  0%
Obscene:       2%
Threat:        0%
Insult:        0%
Identity Hate: 0%



In [65]:
toxicity_level('How are you')

1/1 [==============================] - 0s 39ms/step
Toxicity levels for 'How are you':
Toxic:         2%
Severe Toxic:  0%
Obscene:       1%
Threat:        0%
Insult:        0%
Identity Hate: 0%



In [66]:
toxicity_level('people die')

1/1 [==============================] - 0s 36ms/step
Toxicity levels for 'people die':
Toxic:         76%
Severe Toxic:  15%
Obscene:       33%
Threat:        51%
Insult:        34%
Identity Hate: 2%



In [67]:
toxicity_level('i hate you')

1/1 [==============================] - 0s 58ms/step
Toxicity levels for 'i hate you':
Toxic:         43%
Severe Toxic:  1%
Obscene:       16%
Threat:        0%
Insult:        10%
Identity Hate: 0%



In [68]:
toxicity_level('kill you')

1/1 [==============================] - 0s 37ms/step
Toxicity levels for 'kill you':
Toxic:         61%
Severe Toxic:  4%
Obscene:       12%
Threat:        40%
Insult:        6%
Identity Hate: 0%



In [69]:
toxicity_level('live love laugh')

1/1 [==============================] - 0s 40ms/step
Toxicity levels for 'live love laugh':
Toxic:         9%
Severe Toxic:  0%
Obscene:       2%
Threat:        0%
Insult:        1%
Identity Hate: 0%



In [70]:
toxicity_level('murder')

1/1 [==============================] - 0s 50ms/step
Toxicity levels for 'murder':
Toxic:         34%
Severe Toxic:  2%
Obscene:       4%
Threat:        26%
Insult:        3%
Identity Hate: 1%



In [71]:
toxicity_level('waste')

1/1 [==============================] - 0s 61ms/step
Toxicity levels for 'waste':
Toxic:         35%
Severe Toxic:  1%
Obscene:       8%
Threat:        0%
Insult:        8%
Identity Hate: 0%



In [72]:
toxicity_level('trash')

1/1 [==============================] - 0s 51ms/step
Toxicity levels for 'trash':
Toxic:         58%
Severe Toxic:  1%
Obscene:       9%
Threat:        0%
Insult:        15%
Identity Hate: 0%



In [73]:
toxicity_level('work')

1/1 [==============================] - 0s 42ms/step
Toxicity levels for 'work':
Toxic:         3%
Severe Toxic:  0%
Obscene:       1%
Threat:        0%
Insult:        0%
Identity Hate: 0%



In [74]:
toxicity_level('cooking')

1/1 [==============================] - 0s 46ms/step
Toxicity levels for 'cooking':
Toxic:         11%
Severe Toxic:  5%
Obscene:       14%
Threat:        0%
Insult:        7%
Identity Hate: 0%



In [75]:
toxicity_level('bastard')

1/1 [==============================] - 0s 41ms/step
Toxicity levels for 'bastard':
Toxic:         99%
Severe Toxic:  18%
Obscene:       87%
Threat:        0%
Insult:        82%
Identity Hate: 0%



In [76]:
toxicity_level('bitch')

1/1 [==============================] - 0s 44ms/step
Toxicity levels for 'bitch':
Toxic:         100%
Severe Toxic:  35%
Obscene:       98%
Threat:        0%
Insult:        91%
Identity Hate: 1%



In [77]:
toxicity_level('idiot')

1/1 [==============================] - 0s 44ms/step
Toxicity levels for 'idiot':
Toxic:         98%
Severe Toxic:  5%
Obscene:       65%
Threat:        0%
Insult:        83%
Identity Hate: 0%



In [78]:
toxicity_level('stupid')

1/1 [==============================] - 0s 43ms/step
Toxicity levels for 'stupid':
Toxic:         95%
Severe Toxic:  3%
Obscene:       42%
Threat:        0%
Insult:        62%
Identity Hate: 0%



In [79]:
toxicity_level('COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK')

1/1 [==============================] - 0s 46ms/step
Toxicity levels for 'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK':
Toxic:         44%
Severe Toxic:  1%
Obscene:       23%
Threat:        0%
Insult:        7%
Identity Hate: 0%



In [80]:
toxicity_level('laugh')

1/1 [==============================] - 0s 41ms/step
Toxicity levels for 'laugh':
Toxic:         7%
Severe Toxic:  0%
Obscene:       2%
Threat:        0%
Insult:        1%
Identity Hate: 0%



In [81]:
toxicity_level('sleep')

1/1 [==============================] - 0s 50ms/step
Toxicity levels for 'sleep':
Toxic:         17%
Severe Toxic:  0%
Obscene:       5%
Threat:        0%
Insult:        2%
Identity Hate: 0%

